In [1]:
import pandas as pd
pd.set_option('display.max_columns',100)

In [2]:
df = pd.read_csv(r'C:\Users\srr38\Projects\Data\recommender\movies_metadata.csv')

c:\users\srr38\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,Fa

Metric for scoring : Weighted Rating(WR)=(v*R/(v+m) + m*C/(v+m)) where 
>v is the number of votes garnered by the movie,
m is the minimum number of votes required for the movie to be in the chart (the prerequisite),
R is the mean rating of the movie and
C is the mean rating of all the movies in the dataset


In [4]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

>Calculate the value of m:
based on trial and error or a good guess,
like for a movie to be considered in the rankings, it must have garnered more votes than at least 80% of the movies present in our dataset

In [5]:
#Calculate the number of votes garnered by the 80th percentile movie
m = df['vote_count'].quantile(0.80)
m

50.0

>We can see that only 20% of the movies have gained more than 50 votes. Therefore, our value of m is 50.


>filtering out movies with runtime between 45 mins and 300 mins

In [6]:
movies_sel = df[(df['runtime']>=45) & (df['runtime']<=300)]
movies_sel.shape

(41930, 24)

>filtering movies that have garnered more than m votes

In [7]:
movies_sel = movies_sel[movies_sel['vote_count']>=m]
movies_sel.shape

(8963, 24)

>calculating C: mean rating of all the movies in the dataset

In [8]:
C = movies_sel['vote_average'].mean()
C

6.351500613633828

> calculating scoring metric,WR

In [9]:
m = 50
C = 6.352

def weighted_rating(df,m,C):
    v = df['vote_count']
    R = df['vote_average']
    return (v*R/(v+m)+m*C/(v+m))

movies_sel['score'] = movies_sel.apply(lambda x: weighted_rating(x,m,C),axis=1)

In [10]:
movies_sel_final = movies_sel[['title','vote_count','vote_average','score']]

In [11]:
movies_sel_final = movies_sel_final.sort_values(['score'],ascending=False)

In [12]:
movies_sel_final

,title,vote_count,vote_average,score
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.906751
314,The Shawshank Redemption,8358.0,8.5,8.487226
834,The Godfather,6024.0,8.5,8.482318
40251,Your Name.,1030.0,8.5,8.400556
12481,The Dark Knight,12269.0,8.3,8.292094
...,...,...,...,...
9651,Alone in the Dark,174.0,3.1,3.825893
12911,Disaster Movie,250.0,3.1,3.642000
11557,Epic Movie,334.0,3.2,3.610417
3471,Battlefield Earth,259.0,3.0,3.542395


>Here we see that DDLJ has highest score but if we compare with TSR vote count is very less,
clearly indicating to choose a higher m value

>Let us choose m = 1000 and see the results

In [13]:
m = 1000
C = 6.352
movies_sel['score'] = movies_sel.apply(lambda x: weighted_rating(x,m,C),axis=1)
movies_sel_final = movies_sel[['title','vote_count','vote_average','score']]
movies_sel_final = movies_sel_final.sort_values(['score'],ascending=False)
movies_sel_final.head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.270464
834,The Godfather,6024.0,8.5,8.194191
12481,The Dark Knight,12269.0,8.3,8.153192
2843,Fight Club,9678.0,8.3,8.117569
292,Pulp Fiction,8670.0,8.3,8.098552
351,Forrest Gump,8147.0,8.2,7.997967
15480,Inception,14075.0,8.1,7.984046
22879,Interstellar,11187.0,8.1,7.956568
522,Schindler's List,4436.0,8.3,7.941648
23673,Whiplash,4376.0,8.3,7.937649


## Knowledge Based Recommender

- [x] Ask the user for the genres of movies he/she is looking for
- [x] Ask the user for the duration
- [x] Ask the user for the timeline of the movies recommended
- [x] Using the information collected, recommend movies to the user that have a high weighted rating (according to the IMDB formula) and that satisfy the preceding conditions


In [14]:
import numpy as np
df = df[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

In [15]:
#Data Wrangling
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
df.head(10)

c:\users\srr38\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\srr38\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,title,genres,release_date,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0,1995
5,Heat,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",1995-12-15,170.0,7.7,1886.0,1995
6,Sabrina,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",1995-12-15,127.0,6.2,141.0,1995
7,Tom and Huck,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1995-12-22,97.0,5.4,45.0,1995
8,Sudden Death,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1995-12-22,106.0,5.5,174.0,1995
9,GoldenEye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",1995-11-16,130.0,6.6,1194.0,1995


In [16]:
def convert_int(x):
    try:
        return int(x)
    except:
        return 0
    
df['year'] = df['year'].apply(convert_int)

c:\users\srr38\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [17]:
df = df.drop(columns=['release_date'],axis=1)
df

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995
...,...,...,...,...,...,...
45461,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",90.0,4.0,1.0,0
45462,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",360.0,9.0,3.0,2011
45463,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",90.0,3.8,6.0,2003
45464,Satan Triumphant,[],87.0,0.0,0.0,1917


In [18]:
#Print genres of the first movie
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [19]:

from ast import literal_eval
#Convert all NaN into stringified empty lists
df['genres'] = df['genres'].fillna('[]')

#Apply literal_eval to convert to the list object
df['genres'] = df['genres'].apply(literal_eval)

In [20]:
#Convert list of dictionaries to a list of strings
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995


In [21]:
#Create a new feature by exploding genres
genres = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

c:\users\srr38\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [22]:
genres.name = 'genre'
#Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
gen_df = df.drop('genres', axis=1).join(genres)

In [23]:
gen_df

,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,Animation
0,Toy Story,81.0,7.7,5415.0,1995,Comedy
0,Toy Story,81.0,7.7,5415.0,1995,Family
1,Jumanji,104.0,6.9,2413.0,1995,Adventure
1,Jumanji,104.0,6.9,2413.0,1995,Fantasy
...,...,...,...,...,...,...
45463,Betrayal,90.0,3.8,6.0,2003,Action
45463,Betrayal,90.0,3.8,6.0,2003,Drama
45463,Betrayal,90.0,3.8,6.0,2003,Thriller
45464,Satan Triumphant,87.0,0.0,0.0,1917,NaN


In [24]:
def build_chart(gen_df, percentile=0.8):
    #Ask for preferred genres
    print("Input preferred genre")
    genre = input()
    
    #Ask for lower limit of duration
    print("Input shortest duration")
    low_time = int(input())
    
    #Ask for upper limit of duration
    print("Input longest duration")
    high_time = int(input())
    
    #Ask for lower limit of timeline
    print("Input earliest year")
    low_year = int(input())
    
    #Ask for upper limit of timeline
    print("Input latest year")
    high_year = int(input())
    
    #Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
    movies = gen_df.copy()
    
    #Filter based on the condition

    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

In [26]:
build_chart(gen_df,0.8)

Input preferred genre
Animation
Input shortest duration
30
Input longest duration
100
Input earliest year
1990
Input latest year
2020


,title,runtime,vote_average,vote_count,year,genre,score
359,The Lion King,89.0,8.0,5520.0,1994,Animation,7.925104
30315,Inside Out,94.0,7.9,6737.0,2015,Animation,7.841546
13724,Up,96.0,7.8,7048.0,2009,Animation,7.747266
12704,WALL·E,98.0,7.8,6439.0,2008,Animation,7.742454
0,Toy Story,81.0,7.7,5415.0,1995,Animation,7.636180
...,...,...,...,...,...,...,...
5346,Stuart Little 2,78.0,5.4,632.0,2002,Animation,5.607046
14562,Alvin and the Chipmunks: The Squeakquel,88.0,5.3,688.0,2009,Animation,5.519926
19077,Pocahontas II: Journey to a New World,72.0,4.9,297.0,1998,Animation,5.458231
11055,Garfield: A Tail of Two Kitties,78.0,5.1,477.0,2006,Animation,5.451092


In [29]:
df.to_csv(r"C:\Users\srr38\Projects\Data\recommender\clean_data.csv",index=False)